In [5]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point


# NYC Food Access — Neighborhood-Level (NTA2020) Analysis

This notebook focuses on a **single standardized neighborhood unit**: NYC Neighborhood
Tabulation Areas (NTA2020). We assume the **metrics CSV (with `nta2020` codes)** is the
source of truth for:
- Which neighborhoods exist
- Their IDs
- Metrics like supply gap, food insecurity, unemployment, etc.

In [6]:
CENSUS_TRACTS_GEOJSON = "raw/census_tracts.geojson"   # e.g., 2020 Census tracts GeoJSON/GeoPackage/SHAPE
PANTRIES_PATH      = "data/tabular/pantries.csv"    # can be a GeoJSON of points OR a CSV with lat/lon columns
PANTRIES_GEO = "data/geo/pantries.geojson"
SUPPLY_GAP_CSV     = "raw/Emergency_Food_Supply_Gap_20251110.csv"                    # e.g., "data/supply_gap_by_neighborhood.csv" or None
POPULATION = "data/tabular/nyc_population.csv"

In [7]:

import pandas as pd
import geopandas as gpd
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 60)
pd.set_option("display.width", 140)

# If PANTRIES_PATH is CSV, specify columns:
PANTRY_LAT_COL = "latitude"
PANTRY_LON_COL = "longitude"

CRS_LATLON    = "EPSG:4326"
CRS_PROJECTED = "EPSG:6539"  # NYC-appropriate projected CRS


In [8]:
population = pd.read_csv(POPULATION)   
tracts = gpd.read_file(CENSUS_TRACTS_GEOJSON)  # or your cleaned tract GeoJSON
# make sure it has at least ['geoid','nta2020','ntaname']
pantries = pd.read_csv(PANTRIES_PATH)
metrics = pd.read_csv(SUPPLY_GAP_CSV)

# 3️⃣ Turn pantries into a GeoDataFrame
gpantries = gpd.GeoDataFrame(
    pantries,
    geometry=[Point(xy) for xy in zip(pantries["lng"], pantries["lat"])],
    crs="EPSG:4326"   # WGS84 coordinates
)

# 4️⃣ Ensure coordinate systems match
tracts = tracts.to_crs(epsg=4326)
tracts.shape

(2325, 18)

In [9]:
population.rename(columns={"CensusTract": "geoid"}, inplace=True)
population['geoid'] = population['geoid'].astype(int)
tracts['geoid'] = tracts['geoid'].astype(int)

In [10]:
tracts = tracts.merge(population, on="geoid", how="left")

In [11]:
tracts.to_csv("data/model/tracts_table.csv")

## 1. Aggregate NTA tracts

In [12]:
# Rename for consistency
metrics = metrics.rename(columns={
    "Neighborhood Tabulation Area NTA)": "nta2020",
    "Neighborhood Tabulation Area (NTA) Name": "ntaname"
})

In [13]:
metrics.head()

,Year,nta2020,ntaname,Supply Gap (lbs.),Food Insecure Percentage,Unemployment Rate,Vulnerable Population Score,Weighted Score,Rank
0,2025,BX0401,Concourse-Concourse Village,"-102,142.62426294",26.29%,1053.82%,0.57,5.133096,161.0
1,2025,BX0303,Crotona Park East,"-333,492.64867191",27.24%,1308.90%,0.55,5.628794,141.0
2,2025,MN1102,East Harlem (North),"527,499.52610583",26.68%,1055.74%,0.47,5.908840,121.0
3,2025,BX0602,Tremont,"-113,652.81465869",28.96%,1214.11%,0.53,5.711963,134.0
4,2025,BK1204,Mapleton-Midwood (West),"451,455.520860002",16.87%,564.74%,0.61,5.731608,132.0


In [14]:
metrics["nta2020"] = metrics["nta2020"].str.strip().str.upper()
metrics["Year"] = pd.to_datetime(metrics["Year"], format='%Y')

idx = metrics.groupby("nta2020")["Year"].idxmax()
metrics = metrics.loc[idx].reset_index(drop=True)

In [15]:
# 1️⃣ Keep only the NTA geometries from tracts (since each tract already has nta2020)
nta_geoms = (tracts[['nta2020', 'geoid','geometry']]
             .dissolve(by='nta2020')        # merge all tracts for same nta2020
             .reset_index())

print(f"Unique NTAs (from tracts): {nta_geoms.shape[0]}")

Unique NTAs (from tracts): 262


## Merge metrics and nta geometry gap

In [16]:
# 2️⃣ Join metrics onto those polygons
nta_enriched = nta_geoms.merge(metrics, on='nta2020', how='left')
print(f"NTA polygons after merge: {nta_enriched.shape}")

nta_enriched["Supply Gap (lbs.)"].isna().sum()

NTA polygons after merge: (262, 11)


np.int64(65)

In [17]:
nta_enriched.head()

,nta2020,geometry,geoid,Year,ntaname,Supply Gap (lbs.),Food Insecure Percentage,Unemployment Rate,Vulnerable Population Score,Weighted Score,Rank
0,BK0101,"POLYGON ((-73.95165 40.72349, -73.9525 40.7232...",36047056301,2025-01-01,Greenpoint,"1,153,881.91747087",15.74%,741.51%,0.36,6.811714,44.0
1,BK0102,"POLYGON ((-73.96485 40.70745, -73.96593 40.707...",36047051300,2025-01-01,Williamsburg,"1,011,421.07761282",16.48%,694.02%,0.38,6.326930,75.0
2,BK0103,"POLYGON ((-73.95137 40.69963, -73.95154 40.699...",36047050900,2025-01-01,South Williamsburg,"2,090,950.72621075",27.58%,967.98%,0.60,8.033649,5.0
3,BK0104,"POLYGON ((-73.94042 40.70108, -73.94193 40.700...",36047048900,2025-01-01,East Williamsburg,"1,224,484.03196005",21.34%,739.65%,0.42,6.832984,43.0
4,BK0201,"POLYGON ((-73.99045 40.69372, -73.99059 40.693...",36047000301,2025-01-01,Brooklyn Heights,"322,651.410235809",10.05%,407.41%,0.44,5.637125,139.0


In [18]:
nta_enriched.dropna(subset=['Supply Gap (lbs.)'], inplace=True)

## Pantries per tract and neighborhood

In [19]:
# 3️⃣ Spatially assign pantries to NTA polygons
pantries_in_nta = gpd.sjoin(gpantries, nta_enriched, how='left', predicate='within')
pantry_counts = (pantries_in_nta
                 .groupby('nta2020')
                 .size()
                 .rename('pantry_count')
                 .reset_index())

In [20]:
nta_enriched = nta_enriched.merge(pantry_counts, on='nta2020', how='left')
nta_enriched['pantry_count'] = nta_enriched['pantry_count'].fillna(0).astype(int)

In [21]:
# Export
nta_enriched.drop(columns=['geometry']).to_csv('data/tabular/nta_with_metrics.csv', index=False)
nta_enriched.to_file('data/geo/nta_with_metrics.geojson', driver='GeoJSON')

In [22]:
# Centroids per neighborhood
nta_proj = nta_enriched.to_crs(CRS_PROJECTED)

# 2. Compute centroids in projected CRS
nta_proj["centroid_geom"] = nta_proj.geometry.centroid

# 3. Convert centroids back to WGS84 lat/lon for mapping/export
nta_centroids_ll = nta_proj.set_geometry("centroid_geom").to_crs(CRS_LATLON)

# 4. Extract simple coordinates
nta_centroids_ll["centroid_lon"] = nta_centroids_ll.geometry.x
nta_centroids_ll["centroid_lat"] = nta_centroids_ll.geometry.y

nta_centroids_ll.head()

,nta2020,geometry,geoid,Year,ntaname,Supply Gap (lbs.),Food Insecure Percentage,Unemployment Rate,Vulnerable Population Score,Weighted Score,Rank,pantry_count,centroid_geom,centroid_lon,centroid_lat
0,BK0101,"POLYGON ((997652.824 202865.648, 997417.375 20...",36047056301,2025-01-01,Greenpoint,"1,153,881.91747087",15.74%,741.51%,0.36,6.811714,44.0,1,POINT (-73.94952 40.7295),-73.949516,40.729500
1,BK0102,"POLYGON ((993995.781 197022.868, 993695.322 19...",36047051300,2025-01-01,Williamsburg,"1,011,421.07761282",16.48%,694.02%,0.38,6.326930,75.0,1,POINT (-73.95862 40.71491),-73.958621,40.714914
2,BK0103,"POLYGON ((997733.467 194175.636, 997687.83 194...",36047050900,2025-01-01,South Williamsburg,"2,090,950.72621075",27.58%,967.98%,0.60,8.033649,5.0,1,POINT (-73.95666 40.70324),-73.956659,40.703242
3,BK0104,"POLYGON ((1000769.963 194703.194, 1000351.303 ...",36047048900,2025-01-01,East Williamsburg,"1,224,484.03196005",21.34%,739.65%,0.42,6.832984,43.0,1,POINT (-73.93649 40.7133),-73.936488,40.713299
4,BK0201,"POLYGON ((986897.487 192017.706, 986860.041 19...",36047000301,2025-01-01,Brooklyn Heights,"322,651.410235809",10.05%,407.41%,0.44,5.637125,139.0,0,POINT (-73.99483 40.69547),-73.994830,40.695465


In [23]:
nta_proj = nta_enriched.to_crs(CRS_PROJECTED)
centroids = nta_proj.copy()
centroids['geometry'] = nta_proj.centroid

centroids.head()

,nta2020,geometry,geoid,Year,ntaname,Supply Gap (lbs.),Food Insecure Percentage,Unemployment Rate,Vulnerable Population Score,Weighted Score,Rank,pantry_count
0,BK0101,POINT (998241.957 205056.938),36047056301,2025-01-01,Greenpoint,"1,153,881.91747087",15.74%,741.51%,0.36,6.811714,44.0,1
1,BK0102,POINT (995721.016 199741.715),36047051300,2025-01-01,Williamsburg,"1,011,421.07761282",16.48%,694.02%,0.38,6.326930,75.0,1
2,BK0103,POINT (996267.118 195489.486),36047050900,2025-01-01,South Williamsburg,"2,090,950.72621075",27.58%,967.98%,0.60,8.033649,5.0,1
3,BK0104,POINT (1001857.171 199157.092),36047048900,2025-01-01,East Williamsburg,"1,224,484.03196005",21.34%,739.65%,0.42,6.832984,43.0,1
4,BK0201,POINT (985683.594 192653.281),36047000301,2025-01-01,Brooklyn Heights,"322,651.410235809",10.05%,407.41%,0.44,5.637125,139.0,0


In [24]:
centroids.to_csv("data/model/nta_table.csv", index=False)

### New Pantries Points

In [25]:
# ---- Load pantry locations ----
pantry_path = Path(PANTRIES_GEO)
assert pantry_path.exists(), f"Pantry file not found: {pantry_path}"


pantries = gpd.read_file(pantry_path)
if pantries.crs is None:
    print("⚠️ Pantries have no CRS; assuming WGS84")
    pantries = pantries.set_crs(CRS_LATLON)

pantries_ll = pantries.to_crs(CRS_LATLON)
pantries_proj = pantries_ll.to_crs(CRS_PROJECTED)

print("Pantries:", pantries_ll.shape)
pantries_ll.head(2)

Pantries: (515, 125)


,FID,type_fp,type_sk,program,org_phone,distadd,distboro,distzip,dist_location_info,fp_days_orig,fp_hours_orig,sk_days_orig,sk_hours_orig,fp_mon,fp_mon_open1,fp_mon_close1,fp_mon_open2,fp_mon_close2,fp_tue,fp_tue_open1,fp_tue_close1,fp_tue_open2,fp_tue_close2,fp_wed,fp_wed_open1,fp_wed_close1,fp_wed_open2,fp_wed_close2,fp_thu,fp_thu_open1,...,program_type,fp_mon_open3,fp_mon_close3,fp_tue_open3,fp_tue_close3,fp_wed_open3,fp_wed_close3,fp_thu_open3,fp_thu_close3,fp_fri_open3,fp_fri_close3,fp_sat_open3,fp_sat_close3,fp_sun_open3,fp_sun_close3,sk_mon_open3,sk_mon_close3,sk_tue_open3,sk_tue_close3,sk_wed_open3,sk_wed_close3,sk_thu_open3,sk_thu_close3,sk_fri_open3,sk_fri_close3,sk_sat_open3,sk_sat_close3,sk_sun_open3,sk_sun_close3,geometry
0,1,FP,None,CHURCH OF ST. NICHOLAS OF TOLENTINE,(718) 295-6800,"2345 University Ave, Bronx, New York, 10468",BX,10468,BASEMENT FORDHAM RD ENTRANCE,TUE,9-11AM,None,None,closed,None,None,None,None,open,09:00 AM,11:00 AM,None,None,closed,None,None,None,None,closed,None,...,FP,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,POINT (-73.90567 40.86222)
1,2,FP,,BREAD OF LIFE FOOD PANTRY,(347) 235-3723,"1104 Elder Ave, Bronx, New York, 10472",BX,10472,#15,SAT,1:30 - 4:30PM,,,closed,,,,,closed,,,,,closed,,,,,closed,,...,FP,,,,,,,,,,,,,,,,,,,,,,,,,,,,,POINT (-73.87854 40.82642)


In [26]:
tracts_for_join = tracts[['geoid', "nta2020", "geometry"]].set_geometry("geometry")

pantries_with_geo_ids = gpd.sjoin(
    pantries_ll,
    tracts_for_join,
    how="left",
    predicate="within"
)

# Clean up the sjoin artifact
pantries_with_geo_ids = pantries_with_geo_ids.drop(columns=["index_right"], errors="ignore")

pantries_with_geo_ids.head()

,FID,type_fp,type_sk,program,org_phone,distadd,distboro,distzip,dist_location_info,fp_days_orig,fp_hours_orig,sk_days_orig,sk_hours_orig,fp_mon,fp_mon_open1,fp_mon_close1,fp_mon_open2,fp_mon_close2,fp_tue,fp_tue_open1,fp_tue_close1,fp_tue_open2,fp_tue_close2,fp_wed,fp_wed_open1,fp_wed_close1,fp_wed_open2,fp_wed_close2,fp_thu,fp_thu_open1,...,fp_mon_close3,fp_tue_open3,fp_tue_close3,fp_wed_open3,fp_wed_close3,fp_thu_open3,fp_thu_close3,fp_fri_open3,fp_fri_close3,fp_sat_open3,fp_sat_close3,fp_sun_open3,fp_sun_close3,sk_mon_open3,sk_mon_close3,sk_tue_open3,sk_tue_close3,sk_wed_open3,sk_wed_close3,sk_thu_open3,sk_thu_close3,sk_fri_open3,sk_fri_close3,sk_sat_open3,sk_sat_close3,sk_sun_open3,sk_sun_close3,geometry,geoid,nta2020
0,1,FP,None,CHURCH OF ST. NICHOLAS OF TOLENTINE,(718) 295-6800,"2345 University Ave, Bronx, New York, 10468",BX,10468,BASEMENT FORDHAM RD ENTRANCE,TUE,9-11AM,None,None,closed,None,None,None,None,open,09:00 AM,11:00 AM,None,None,closed,None,None,None,None,closed,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,POINT (-73.90567 40.86222),36005025500,BX0701
1,2,FP,,BREAD OF LIFE FOOD PANTRY,(347) 235-3723,"1104 Elder Ave, Bronx, New York, 10472",BX,10472,#15,SAT,1:30 - 4:30PM,,,closed,,,,,closed,,,,,closed,,,,,closed,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,POINT (-73.87854 40.82642),36005005002,BX0901
2,4,FP,,CHANCE FOR CHILDREN,(347) 616-3228,"11 Mc Keever Pl, Brooklyn, New York, 11225",BK,11225,1ST FLOOR (BASEMENT LEVEL),FRI,10AM-12:30PM,,,closed,,,,,closed,,,,,closed,,,,,closed,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,POINT (-73.95848 40.66569),36047032500,BK0901
3,6,FP,None,HEALTH ESSENTIAL ASSOCIATION INC (BK),(646) 515-6898,"2101 E 16th St, Brooklyn, New York, 11229",BK,11229,2ND FLOOR,FRI (4TH),10AM-12PM,None,None,closed,None,None,None,None,closed,None,None,None,None,closed,None,None,None,None,closed,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,POINT (-73.95528 40.59861),36047058000,BK1502
4,10,FPHA,None,AIDS CENTER OF QUEENS COUNTY (WOODSIDE),(718) 472-9400,"62-07 Woodside Ave, Woodside, New York, 11377",QN,11377,3RD FLOOR,"TUE, THUR",9-11AM,None,None,closed,None,None,None,None,open,08:00 AM,10:00 AM,None,None,closed,None,None,None,None,closed,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,POINT (-73.90219 40.74466),36081026100,QN0203


In [27]:
pantries_with_geo_ids['id'] = pantries_with_geo_ids.index

In [28]:
pantries_with_geo_ids

,FID,type_fp,type_sk,program,org_phone,distadd,distboro,distzip,dist_location_info,fp_days_orig,fp_hours_orig,sk_days_orig,sk_hours_orig,fp_mon,fp_mon_open1,fp_mon_close1,fp_mon_open2,fp_mon_close2,fp_tue,fp_tue_open1,fp_tue_close1,fp_tue_open2,fp_tue_close2,fp_wed,fp_wed_open1,fp_wed_close1,fp_wed_open2,fp_wed_close2,fp_thu,fp_thu_open1,...,fp_tue_open3,fp_tue_close3,fp_wed_open3,fp_wed_close3,fp_thu_open3,fp_thu_close3,fp_fri_open3,fp_fri_close3,fp_sat_open3,fp_sat_close3,fp_sun_open3,fp_sun_close3,sk_mon_open3,sk_mon_close3,sk_tue_open3,sk_tue_close3,sk_wed_open3,sk_wed_close3,sk_thu_open3,sk_thu_close3,sk_fri_open3,sk_fri_close3,sk_sat_open3,sk_sat_close3,sk_sun_open3,sk_sun_close3,geometry,geoid,nta2020,id
0,1,FP,None,CHURCH OF ST. NICHOLAS OF TOLENTINE,(718) 295-6800,"2345 University Ave, Bronx, New York, 10468",BX,10468,BASEMENT FORDHAM RD ENTRANCE,TUE,9-11AM,None,None,closed,None,None,None,None,open,09:00 AM,11:00 AM,None,None,closed,None,None,None,None,closed,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,POINT (-73.90567 40.86222),36005025500,BX0701,0
1,2,FP,,BREAD OF LIFE FOOD PANTRY,(347) 235-3723,"1104 Elder Ave, Bronx, New York, 10472",BX,10472,#15,SAT,1:30 - 4:30PM,,,closed,,,,,closed,,,,,closed,,,,,closed,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,POINT (-73.87854 40.82642),36005005002,BX0901,1
2,4,FP,,CHANCE FOR CHILDREN,(347) 616-3228,"11 Mc Keever Pl, Brooklyn, New York, 11225",BK,11225,1ST FLOOR (BASEMENT LEVEL),FRI,10AM-12:30PM,,,closed,,,,,closed,,,,,closed,,,,,closed,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,POINT (-73.95848 40.66569),36047032500,BK0901,2
3,6,FP,None,HEALTH ESSENTIAL ASSOCIATION INC (BK),(646) 515-6898,"2101 E 16th St, Brooklyn, New York, 11229",BK,11229,2ND FLOOR,FRI (4TH),10AM-12PM,None,None,closed,None,None,None,None,closed,None,None,None,None,closed,None,None,None,None,closed,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,POINT (-73.95528 40.59861),36047058000,BK1502,3
4,10,FPHA,None,AIDS CENTER OF QUEENS COUNTY (WOODSIDE),(718) 472-9400,"62-07 Woodside Ave, Woodside, New York, 11377",QN,11377,3RD FLOOR,"TUE, THUR",9-11AM,None,None,closed,None,None,None,None,open,08:00 AM,10:00 AM,None,None,closed,None,None,None,None,closed,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,POINT (-73.90219 40.74466),36081026100,QN0203,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,815,FP,None,FIRST UNITED CHRISTIAN CHURCH (FP),None,"109 Victory Blvd, Staten Island, NY, 10301, USA",SI,10301,None,None,None,None,None,closed,None,None,None,None,open,10:00 AM,01:00 PM,None,None,closed,None,None,None,None,closed,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,POINT (-74.08022 40.63766),36085001100,SI0101,510
511,816,FP,None,MAS STATEN ISLAND CENTER FOOD PANTRY (FP),(800) 668-0742,"180 Burgher Ave, Staten Island, NY, 10304, USA",SI,10304,None,None,None,None,None,closed,None,None,None,None,closed,None,None,None,None,closed,None,None,None,None,closed,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,POINT (-74.09192 40.59242),36085009602,SI0201,511
512,817,FP,None,MASJID AR RAHMAN FOOD PANTRY,(718) 740-5025,"98-10 211th St, Queens Village, NY, 11429, USA",QN,11429,None,None,None,None,None,closed,None,None,None,None,closed,None,None,None,None,closed,None,None,None,None,closed,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,No

In [29]:
pantries_with_geo_ids.to_csv("data/model/pantries_table.csv", index=False)

In [30]:
TRACT_PATH = "data/model/tract_table.csv"
METRICS    = "data/tabular/nta_with_metrics.csv"                    # e.g., "data/supply_gap_by_neighborhood.csv" or None
NTA_PATH = "data/model/nta_table.csv"
PANTRIES_PATH = "data/model/pantries_table.csv" 

In [31]:
# If PANTRIES_PATH is CSV, specify columns:
PANTRY_LAT_COL = "lat"
PANTRY_LON_COL = "lng"

CRS_LATLON    = "EPSG:4326"
CRS_PROJECTED = "EPSG:6539"  # NYC-appropriate projected CRS

In [32]:
# A: Food pantries
pantries = pd.read_csv(PANTRIES_PATH)

NUM_PANTRIES = pantries.shape[0]
print(f"Number of food pantries: {NUM_PANTRIES}")

Number of food pantries: 515


In [33]:
metrics_by_nta = pd.read_csv(NTA_PATH)
NUM_NEIGHBORHOODS = metrics_by_nta.shape[0]
print(f"Number of neighborhoods: {NUM_NEIGHBORHOODS}")

Number of neighborhoods: 197


In [71]:
# B: Supply gap by neighborhood
gap_per_nta= metrics_by_nta[['nta2020','ntaname','geoid','Supply Gap (lbs.)']]
gap_per_nta

,nta2020,ntaname,geoid,Supply Gap (lbs.)
0,BK0101,Greenpoint,36047056301,"1,153,881.91747087"
1,BK0102,Williamsburg,36047051300,"1,011,421.07761282"
2,BK0103,South Williamsburg,36047050900,"2,090,950.72621075"
3,BK0104,East Williamsburg,36047048900,"1,224,484.03196005"
4,BK0201,Brooklyn Heights,36047000301,"322,651.410235809"
...,...,...,...,...
192,SI0301,Oakwood-Richmondtown,36085012805,"529,757.683825265"
193,SI0302,Great Kills-Eltingville,36085014608,"1,091,230.79111141"
194,SI0303,Arden Heights-Rossville,36085017007,"600,174.677844235"
195,SI0304,Annadale-Huguenot-Prince's Bay-Woodrow,36085017009,"620,825.608054139"


In [72]:
# C: Pantries per NTA:
pantries_per_nta = metrics_by_nta[['nta2020','pantry_count', 'ntaname']]
pantries_per_nta

,nta2020,pantry_count,ntaname
0,BK0101,1,Greenpoint
1,BK0102,1,Williamsburg
2,BK0103,1,South Williamsburg
3,BK0104,1,East Williamsburg
4,BK0201,0,Brooklyn Heights
...,...,...,...
192,SI0301,0,Oakwood-Richmondtown
193,SI0302,1,Great Kills-Eltingville
194,SI0303,0,Arden Heights-Rossville
195,SI0304,2,Annadale-Huguenot-Prince's Bay-Woodrow


In [75]:
pantries_table = pantries[['id', 'program',  'geoid', 'nta2020', 'geometry']]

In [76]:
tracts = pd.read_csv("data/model/tracts_table.csv")
tracts

,Unnamed: 0,:id,:version,:created_at,:updated_at,ctlabel,borocode,boroname,ct2020,boroct2020,cdeligibil,ntaname,nta2020,cdta2020,cdtaname,geoid,shape_leng,shape_area,geometry,County,Borough,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Citizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,0,row-n3v9_fibw~s7xp,rv-wmam~mr9u-utmz,2025-08-21 13:56:10.254000+00:00,2025-08-21 13:56:10.254000+00:00,1.00,1,Manhattan,100,1000100,I,The Battery-Governors Island-Ellis Island-Libe...,MN0191,MN01,MN01 Financial District-Tribeca (CD 1 Equivalent),36061000100,10833.043929,1.843005e+06,MULTIPOLYGON (((-74.04387761639944 40.69018767...,New York,Manhattan,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
1,1,row-r3ub~566h_ntm6,rv-u9kx_vkcu~bxxk,2025-08-21 13:56:10.254000+00:00,2025-08-21 13:56:10.254000+00:00,14.01,1,Manhattan,1401,1001401,I,Lower East Side,MN0302,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061001401,5075.332000,1.006117e+06,MULTIPOLYGON (((-73.9883662631772 40.716445702...,New York,Manhattan,3155.0,1488.0,1667.0,11.8,65.9,3.5,0.0,15.5,2412.0,76250.0,14590.0,64817.0,8087.0,5.7,10.8,72.5,6.7,18.6,0.0,2.1,4.1,1.5,60.1,15.7,10.4,8.2,29.0,1734.0,77.6,11.3,10.1,1.0,1.8
2,2,row-7ax4_er4v-39ht,rv-njx8.gskr.fd2m,2025-08-21 13:56:10.254000+00:00,2025-08-21 13:56:10.254000+00:00,14.02,1,Manhattan,1402,1001402,E,Lower East Side,MN0302,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061001402,4459.156019,1.226206e+06,MULTIPOLYGON (((-73.98507342254645 40.71908329...,New York,Manhattan,2932.0,1315.0,1617.0,22.7,32.8,6.2,0.0,32.1,2267.0,24375.0,3314.0,28093.0,4682.0,31.0,47.2,48.3,27.9,18.6,1.5,3.7,3.0,0.3,71.4,12.9,4.9,7.6,32.8,1470.0,76.8,9.5,13.7,0.0,5.2
3,3,row-rbey.gfbd-2auc,rv-jq33.3rim~hsfz,2025-08-21 13:56:10.254000+00:00,2025-08-21 13:56:10.254000+00:00,18.00,1,Manhattan,1800,1001800,I,Lower East Side,MN0302,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061001800,6391.921174,2.399277e+06,MULTIPOLYGON (((-73.9898545438136 40.720520352...,New York,Manhattan,8326.0,4741.0,3585.0,9.8,30.7,3.5,0.1,54.5,6174.0,59416.0,7717.0,34216.0,4507.0,26.0,40.5,49.1,20.7,17.4,1.5,11.3,3.8,2.3,55.1,22.5,4.9,11.3,29.8,4951.0,91.2,2.9,5.4,0.6,4.2
4,4,row-2v9z-5b5y_tyj9,rv-vccd_dzy9_exmx,2025-08-21 13:56:10.254000+00:00,2025-08-21 13:56:10.254000+00:00,22.01,1,Manhattan,2201,1002201,E,Lower East Side,MN0302,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061002201,5779.062607,1.740174e+06,MULTIPOLYGON (((-73.97875234984308 40.71993370...,New York,Manhattan,6861.0,3245.0,3616.0,42.7,15.6,15.4,0.0,23.8,5536.0,34988.0,11552.0,26403.0,3845.0,27.7,40.4,42.1,21.6,25.7,3.0,7.6,2.3,1.7,75.1,17.1,2.1,1.7,36.4,3016.0,83.1,9.0,7.9,0.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2320,2320,row-ndza-qu8t.g982,rv-sqbz_2hzx~u37j,2025-08-21 13:56:10.254000+00:00,2025-08-21 13:56:10.254000+00:00,99.03,1,Manhattan,9903,1009903,I,Chelsea-Hudson Yards,MN0401,MN04,MN04 Chelsea-Hell's Kitchen (CD 4 Approximation),36061009903,10607.853045,4.533960e+06,MULTIPOLYGON (((-73.99729876528028 40.75710704...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2321,2321,row-c8ht_wwjj_2g4i,rv-6zbe-6g93~c64w,2025-08-21 13:56:10.254000+00:00,2025-08-21 13:56:10.254000+00:00,117.00,1,Manhattan,11700,1011700,I,Chelsea-Hudson Yards,MN0401,MN04,MN04 Chelsea-Hell's Kitchen (CD 4 Approximation),36061011700,7692.557894,2.125061e+06,MULTIPOLYGON (((-74.00178824088411 40.76229452...,New York,Manhattan,3870.0,

In [38]:
tracts = tracts[['geoid','nta2020', 'geometry', 'boroname', 'TotalPop']]

In [39]:
nta = pd.read_csv(NTA_PATH)
SUPPLY_GAP_COL = "Supply Gap (lbs.)"
nta_table = nta[['nta2020', 'ntaname','geoid','geometry','Supply Gap (lbs.)','pantry_count',]]
nta_table[SUPPLY_GAP_COL] = nta_table[SUPPLY_GAP_COL].apply(lambda x: x.replace(',', '')).astype(float)

/var/folders/dj/dxzvsp311dl66s42c2nyqbk00000gn/T/ipykernel_81897/1678640100.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nta_table[SUPPLY_GAP_COL] = nta_table[SUPPLY_GAP_COL].apply(lambda x: x.replace(',', '')).astype(float)


In [40]:
EXCESS_NTA = nta_table[nta_table[SUPPLY_GAP_COL] < 0]
DEFICIT_NTA = nta_table[nta_table[SUPPLY_GAP_COL] > 0]

In [41]:
excess_ids = EXCESS_NTA.nta2020
deficit_ids = DEFICIT_NTA.nta2020

In [43]:
print("EXCESS NTAs:", len(EXCESS_NTA))
print("DEFICIT NTAs:", len(DEFICIT_NTA))

EXCESS NTAs: 55
DEFICIT NTAs: 142


In [44]:
pantries_with_geo_ids = gpd.sjoin(
    pantries_ll,
    tracts_for_join,
    how="left",
    predicate="intersects"   # safer than 'within'
)

In [45]:
print(pantries_with_geo_ids['nta2020'].unique()[:20])

['BX0701' 'BX0901' 'BK0901' 'BK1502' 'QN0203' 'QN1401' 'QN1201' 'MN0901'
 'MN1102' 'MN1002' 'MN0903' 'MN0604' 'MN0802' 'MN0702' 'QN1205' 'QN0303'
 'QN0302' 'QN0401' 'QN0801' 'QN0704']


In [46]:
pantries_with_geo_ids['nta2020'] = pantries_with_geo_ids['nta2020'].astype(str).str.strip().str.upper()
EXCESS_NTA_ids = EXCESS_NTA['nta2020'].astype(str).str.strip().str.upper()
DEFICIT_NTA_ids = DEFICIT_NTA['nta2020'].astype(str).str.strip().str.upper()

excess_pantries = pantries_with_geo_ids[
    pantries_with_geo_ids['nta2020'].isin(EXCESS_NTA_ids)
]
deficit_pantries = pantries_with_geo_ids[
    pantries_with_geo_ids['nta2020'].isin(DEFICIT_NTA_ids)
]

In [47]:
print(len(excess_pantries), len(deficit_pantries))

231 283


In [50]:
import haversine

# Add a unique ID to each pantry
excess_pantries = excess_pantries.copy()
deficit_pantries = deficit_pantries.copy()

excess_pantries['id'] = excess_pantries.index.astype(str)
deficit_pantries['id'] = deficit_pantries.index.astype(str)

# Now extract lat/lon as tuples
excess_coords = [(p.y, p.x) for p in excess_pantries['geometry']]
deficit_coords = [(p.y, p.x) for p in deficit_pantries['geometry']]

# Initialize distance matrix
dist_matrix = np.zeros((len(excess_coords), len(deficit_coords)))

# Compute distances using haversine
from haversine import haversine, Unit

for i, e_coord in enumerate(excess_coords):
    for j, d_coord in enumerate(deficit_coords):
        dist_matrix[i, j] = haversine(e_coord, d_coord, unit=Unit.MILES)

# Build DataFrame
dist_df_pantry = pd.DataFrame(
    dist_matrix,
    index=excess_pantries['id'],
    columns=deficit_pantries['id']
)

print(dist_df_pantry.shape)
dist_df_pantry

(231, 283)


id,0,1,2,3,5,7,8,9,10,11,13,14,15,17,18,20,21,22,24,25,26,27,28,33,34,35,39,40,45,49,...,469,472,473,474,477,478,479,480,482,483,485,486,490,491,494,497,498,499,500,501,502,503,504,505,508,509,510,511,513,514
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,8.124483,5.783341,6.201623,10.467823,12.437733,5.620297,4.529374,5.094675,6.063299,3.698567,5.935336,5.182828,4.973688,1.728440,1.812705,1.540874,1.449805,12.431406,4.656210,4.226348,3.007559,6.826279,6.108034,2.961284,5.558988,11.027407,11.731829,5.833639,5.948087,11.150230,...,10.756206,6.417664,4.305011,7.480021,6.473927,7.048698,4.882871,6.665792,6.643754,4.929075,7.629645,12.837141,0.838233,6.106685,10.271816,10.317374,9.307955,0.530382,4.527873,6.109188,5.711591,11.221461,5.540709,5.611690,4.437986,6.572050,11.901670,14.474525,9.444375,1.617911
6,12.275282,9.427942,8.827752,11.022262,7.264138,11.377573,10.185154,10.693116,11.448102,9.778236,11.326234,11.034125,11.052256,5.158138,5.233993,4.693794,4.763358,7.086197,4.222247,1.921443,4.960991,3.515403,8.452918,8.403461,9.319202,12.423270,12.149898,9.515496,6.785859,13.863938,...,13.605902,10.938460,9.509508,11.894475,11.239575,11.342221,9.973186,9.455076,10.203058,8.943949,9.408098,13.863378,5.392088,11.306069,13.125282,4.225597,4.054235,5.947233,7.725938,11.115849,3.700469,15.066667,7.326476,11.261620,10.375681,11.723441,15.496740,17.245925,11.743005,4.666845
12,7.565830,6.424174,6.812850,11.447883,15.811546,3.337970,2.983074,3.390669,4.413094,0.366142,4.316662,2.799205,1.764999,5.171772,5.182814,5.281439,5.196440,15.865325,7.986011,7.966240,5.890083,10.088848,6.971109,2.512878,5.429742,11.390333,12.620142,5.660555,7.840796,10.540923,...,10.905248,5.903278,3.011226,6.784205,5.632693,6.604327,3.419624,7.713941,6.278562,4.853617,9.112265,13.218386,4.561030,4.756597,10.488150,14.039845,13.005749,4.046216,5.476642,5.054502,9.168844,10.460024,7.026577,3.585808,1.312665,5.152441,10.477360,13.476082,9.428587,5.341506
16,14.205805,11.376888,11.354230,12.897497,6.298584,13.891251,12.688975,13.165253,13.828480,12.539981,13.712080,13.589650,13.715171,7.713030,7.764392,7.373590,7.451741,5.944859,5.981808,4.710082,7.710569,5.182693,10.952094,11.204710,11.993529,14.474197,13.909986,12.173566,9.129435,16.206419,...,14.988622,13.058239,12.303296,13.933317,13.423900,13.367447,12.758793,11.125846,12.822015,11.658373,10.723729,15.737214,8.088661,13.628933,14.536103,2.065236,2.315599,8.632046,10.395922,13.382929,4.974975,16.769530,9.809622,13.738289,13.164488,14.016757,17.999945,19.522986,14.060632,7.330217
19,7.213798,4.506699,8.125161,12.141911,12.416536,6.013764,4.811701,5.296196,6.027550,5.323343,5.905637,5.720485,5.981012,0.547518,0.401329,1.413857,1.451804,12.321599,2.717581,3.802000,4.320824,7.202255,7.975708,5.037529,7.623113,12.874704,13.408446,7.895382,7.458935,13.184754,...,9.333063,5.686626,6.384025,6.708584,5.916763,6.190878,6.966126,5.070078,8.707921,7.004962,5.784588,14.650372,1.489028,5.896173,8.838103,9.609607,7.857964,1.581698,6.469768,5.733040,3.895053,10.219024,7.251961,5.868298,6.223353,6.323140,14.002957,16.536076,11.405022,1.320888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,12.747479,9.899016,9.890040,11.904442,7.053819,12.175896,10.975654,11.464230,12.164470,10.764458,12.045650,11.859397,11.953148,5.972242,6.030960,5.603198,5.680154,6.803842,4.535029,2.949954,6.050392,4.240063,9.508593,9.457000,10.406523,13.363935,12.998454,10.601154,7.796556,14.888309,...,13.817921,11.506325,10.581928,12.421820,11.843559,11.860769,11.051634,9.784558,11.283732,10.033707,9.567096,14.752703,6.316503,11.988456,13.348430,3.472786,3.036425,6.862164,8.812253,11.765844,3.733946,15.433451,8.376287,12.036354,11.395548,12.389642,16.559184,18.256485,12.758385,5.562527
496,13.498711,11.009863,3.653903,5.887440,8.349878,10

In [57]:
dist_df_pantry.to_csv("data/model/dist_pantry.csv")

In [ ]:
print("Distance matrix shape:", dist_df_pantry.shape)
print("Index vs excess_ids:", set(dist_df_pantry.index) - set(excess_ids))
print("Columns vs deficit_ids:", set(dist_df_pantry.columns) - set(deficit_ids))

Distance matrix shape: (231, 283)
Index vs excess_ids: {512, 4, 6, 12, 16, 19, 23, 29, 30, 31, 32, 36, 37, 38, 41, 42, 43, 44, 46, 47, 48, 50, 51, 55, 56, 59, 61, 63, 64, 65, 66, 68, 69, 70, 71, 72, 76, 79, 82, 83, 85, 86, 87, 88, 89, 91, 92, 93, 94, 96, 97, 102, 103, 106, 107, 115, 117, 118, 120, 121, 123, 126, 135, 136, 137, 144, 145, 146, 148, 151, 153, 155, 156, 159, 161, 163, 167, 175, 176, 177, 178, 182, 184, 187, 188, 190, 194, 195, 196, 197, 198, 202, 203, 205, 206, 209, 210, 211, 212, 215, 229, 230, 231, 232, 235, 236, 237, 239, 240, 241, 242, 243, 244, 245, 249, 257, 258, 259, 260, 262, 264, 266, 269, 273, 278, 291, 292, 293, 294, 296, 297, 298, 300, 302, 305, 307, 314, 317, 326, 329, 330, 331, 333, 334, 335, 339, 340, 342, 343, 344, 346, 348, 349, 350, 351, 353, 354, 356, 357, 358, 359, 360, 361, 362, 363, 364, 367, 370, 372, 375, 376, 377, 378, 379, 383, 387, 388, 390, 393, 394, 395, 397, 398, 399, 401, 402, 404, 406, 409, 412, 414, 420, 422, 423, 424, 425, 427, 428, 430, 4

In [ ]:
nta_table["Supply Gap (lbs.)"].describe()
print("Negative gaps:", (nta_table["Supply Gap (lbs.)"] < 0).sum())
print("Positive gaps:", (nta_table["Supply Gap (lbs.)"] > 0).sum())

Negative gaps: 55
Positive gaps: 142


In [ ]:
dist_df_pantry.head()
list(supply.keys())[:10]
list(demand.keys())[:10]

['0', '1', '2', '3', '5', '7', '8', '9', '10', '11']